In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn

In [ ]:
# CREATING DATA AND TRAIN TEST SPLIT

# Create *known* parameters
weight = 0.7
bias = 0.3

# Create data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

# Create train/test split
train_split = int(0.8 * len(X)) # 80% of data used for training set, 20% for testing
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)


# Creating the Model

# Creating a linear regression model class
class LinearRegressionModel(nn.Module):
  # Creating constructor
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1, # <- start with random weights (this will get adjusted as the model learns)
                                            dtype = torch.float), # <- PyTorch loves float32 by default
                                            requires_grad = True) # by default is also true
    self.bias = nn.Parameter(torch.randn(1,
                                         dtype = torch.float), # <- PyTorch loves float32 by default
                                          requires_grad = True)

    # Forward method to define the computation in the model
  def forward(self, x: torch.Tensor) -> torch.Tensor: # <- "x" is the input data
    return self.weights * x + self.bias # this is the linear regression formula


torch.manual_seed(42) # we'll get the same initial values when using seed
model_0 = LinearRegressionModel() # calling the class


# Defining the loss function and Optimizer

loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(params = model_0.parameters(),
                            lr = 0.01)

torch.manual_seed(42)

epochs = 100 # hyperparameter

# 1. Loop through the data
for epoch in range(epochs):
  # setting the model in training mode
  model_0.train()  # ----->>>>>>> train mode in PyTorch sets all parameters that require gradients to require gradients

  # 2. Forward pass
  y_pred = model_0(X_train)

  # 3. Calculate the loss
  loss = loss_fn(y_pred, y_train) # predictions, labels
  print(f"Loss: {loss}")

  # 4. Optimizer 0 grad
  optimizer.zero_grad()

  # 5. Perform backpropagation on the loss wrt parameters of the model
  loss.backward()

  # 6. Step the optimizer (perform gradient descent)
  optimizer.step()                  # it accumilates the value that it's supposed to change through the loop


  # Testing
  model_0.eval()  # Evaluation mode: Turns off different settings in the model not needed for evaluation/testing purposes (dropout/batch normalization)

  with torch.inference_mode(): # turns off gradient tracking + other things - to make te code faster
    # 1. Forward pass
    test_pred = model_0(X_test)

    # 2. Calculate loss
    test_loss = loss_fn(test_pred, y_test)

  if epoch %10 ==0: # not printing what's happening in every epoch
    print(f"Epoch: {epoch} | Test: {loss} | {test_loss}")
    print(model_0.state_dict())


# Lesson 53 - Saving a model

There are 3 ways to save a model in PyTorch:
1. `torch.save():` allows to save a PyTorch object in Python's pickle format
2. `torch.load():` allows to load a saved pytorch object
3. `torch.nn.load_state_dict():` allows to load a model's saved state dictionary

In [4]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.5784])), ('bias', tensor([0.3513]))])

- `state_dict():` stores important parameters in a simple python dictionary
- Dictionary that holds the state of the model
- but later on we may be working with millions of parameters, so looking at state_dict may not be easy

## Saving PyTorch Model

In [5]:
from pathlib import Path

# 1. Creating a model directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "01_pytorch_workflow_model_0.pth" # we save using .pt or .pth extension
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

MODEL_SAVE_PATH

PosixPath('models/01_pytorch_workflow_model_0.pth')

In [6]:
torch.save(obj= model_0.state_dict(),
           f=MODEL_SAVE_PATH)

In [7]:
!ls -l models

total 4
-rw-r--r-- 1 root root 1680 Nov 20 07:58 01_pytorch_workflow_model_0.pth


- Here we saved the model's state_dict() (recommended)
- We can also save the entire model

## Loading the Model

- Since we saved the model's state_dict rather than the entire model, we'll create a new instance of our model class and load the save state_dict() into that.
- To load in the state_dict(), we have to instantiate a new instance of our model class

In [8]:
loaded_model_0 = LinearRegressionModel()

In [9]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

- It is iniallized with random parameters

In [10]:
# loading the saved state_dict of model_0
# this will update the new instance with updated parameters

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [12]:
loaded_model_0.state_dict()

OrderedDict([('weights', tensor([0.5784])), ('bias', tensor([0.3513]))])

- Our parameters are now updated

## Making predictions

In [13]:
# for making predictions we are in evaluation mode
loaded_model_0.eval()

with torch.inference_mode():
  loaded_model_preds = loaded_model_0(X_test)

loaded_model_preds

tensor([[0.8141],
        [0.8256],
        [0.8372],
        [0.8488],
        [0.8603],
        [0.8719],
        [0.8835],
        [0.8950],
        [0.9066],
        [0.9182]])